# Lab 8: Variational Autoencoders

Name1, Student's ID1<br>



## Lab Instruction: Generate Random Handwriting Number

In this lab, you will learn do the generative model using variational autoencoder to generate random image.</br>

This is what we are going to do in this lab: https://www.siarez.com/projects/variational-autoencoder </br>
About the variational autoencoder: https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf

This lab, we created model using functional API > https://keras.io/models/model/ 

### Autoencoder Model

![autoencoder](autoencoder.jpg "Autoencoder")

### Variational Autoencoder Model

![VAE](autoencoder.png "Variational Autoencoder")

### Reconstruct image using Variational Autoencoder Model

![variation](variation.png "Variational Autoencoder")


***Images from:<br>***
*1. <a href=https://blog.keras.io/building-autoencoders-in-keras.html> Keras Blog </a></br>*
*2. Manning - Deep Learning with Python Book*


In [ ]:
# Import required libraries
import keras
from keras import models
from keras import layers
from keras import backend as K
from keras import callbacks

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from sklearn.model_selection import train_test_split

%matplotlib inline

### Load MNIST data 

In [ ]:
# Load MNIST data
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Data exploration: Shape
x_train.shape

In [ ]:
# Data exploration: Data type
x_train.dtype

In [ ]:
# Min, Max value
np.min(x_train),np.max(x_train)

### Preprocess data 

In [ ]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_train = x_train/255.

x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)
x_test = x_test/255.

In [ ]:
# shape after preprocess
x_train.shape

### Split Train, Test, Validation data 

In [ ]:
# Split data
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,
                                                  test_size=0.1,
                                                  stratify=y_train,
                                                  random_state=0)

### Create variational autoencoder model 

In technical terms, here’s how a VAE works:
1. An encoder module turns the input samples input_img into two parameters in a latent space of representations, z_mean and z_log_variance.
2. You randomly sample a point z from the latent normal distribution that’s assumed to generate the input image, via
> z= z_mean+exp(z_log_variance)* epsilon, *where epsilon is a random tensor of small values.*
3. A decoder module maps this point in the latent space back to the original input image.

In [ ]:
# Define variables


In [ ]:
# Encoder Model



In [ ]:
# Variation Parameters
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [ ]:
# Sampling distribution funtion

def sampling(arg):
    z_mean, z_log_var = arg
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim),mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

In [ ]:
# Distribution function sampling layer


To define a deconvolution layer
> ```layers.Conv2DTranspose( )```

See: https://keras.io/layers/convolutional/ Search for ```Conv2DTranspose```

In [ ]:
# Decoder model


In [ ]:
# Wrap up encoder model


In [ ]:
# Wrap up decoder model


In [ ]:
# Warp up VAE model


The parameters of a VAE are trained via two loss functions: 
1. A reconstruction loss that forces the decoded samples to match the initial inputs.
2. A regularization loss (The Kullback-Liebler divergence) that helps learn well-formed latent spaces and reduce overfitting to the training data.

In [ ]:
# Create custom loss function

def vae_loss(x, x_decoded_mean):
    xent_loss = K.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

### Compile and Summarise Model

In [ ]:
# Compile and summarise model


In [ ]:
# Plot training history
def viz_loss(history): 
    
    # Print the result from the last epoch
    print('Last Training set loss: %s'%history.history['loss'][-1])
    print('Last Validation set loss: %s'%history.history['val_loss'][-1])
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(loss) + 1)   
    
    plt.plot(epochs, loss, 'c--', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

### Training The VAE Model

In [ ]:
# Define callback for checkpoint 
checkpoint = callbacks.ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)

In [ ]:
# Define callback for Tensorboard
tensorboard = callbacks.TensorBoard(log_dir='/tmp/logs')

To call a Tensorboard, open terminal and type
>``` tensorboard --logdir=/full_path_to_your_logs ```

In [ ]:
# Train model


In [ ]:
# Visualize loss


### Predicting on Test Datast 

In [ ]:
# Reconstruct image using test dataset


In [ ]:
# Show test image

x_test_reshape = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2])
visualize_image(x_test_reshape)

In [ ]:
# Show reconstruct image

decoded_imgs = decoded_imgs.reshape(decoded_imgs.shape[0],decoded_imgs.shape[1],decoded_imgs.shape[2])
visualize_image(decoded_imgs)

### Try Randomly Generate Data

In [ ]:
# Randomly generate a latent vector

n = 15
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

random_input = np.random.uniform(high=2,low=-2,size=(64,latent_dim))

In [ ]:
# Reconstruct image


In [ ]:
# Exploration: X_decode's shape


In [ ]:
# Reshape x_decode
decode_digit = x_decoded.reshape(64,digit_size, digit_size)

### Show Generated Image

In [ ]:
# Define visualized image function

def visualize_image(image, row=3, col=10, title='Generated hadwriting digit image'):
    fig, ax = plt.subplots(row,col,figsize=(13,4))
    fig.suptitle(title)
    for i in range(row):
        for j in range(col):
            ax[i,j].imshow(image[j + i*10],cmap='gray')
            ax[i,j].get_xaxis().set_visible(False)
            ax[i,j].get_yaxis().set_visible(False)

In [ ]:
# Show reconstruct image
visualize_image(decode_digit)

### Scan The Latent Space

In [ ]:
# Display a 2D manifold of the digits

n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# we will sample n points within [-15, 15] standard deviations
grid_x = np.linspace(-15, 15, n)
grid_y = np.linspace(-15, 15, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]]) * 5e-4
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()

### Visualize Latent Space 

The colored clusters is a type of digit. Close clusters are digits that are structurally similar (i.e. digits that share information in the latent space).

In [ ]:
# Estimate latent space from test set


In [ ]:
# Define latent space visualizatio function

def show_latent_space(encoded_imgs):
    plt.figure(figsize=(10, 8))
    plt.scatter(encoded_imgs[:, 0], encoded_imgs[:, 1], 
                c=y_test,alpha=.7, s=3**2, cmap='viridis')
    plt.colorbar()
    plt.show()

In [ ]:
# Show latent space
show_latent_space(x_test_encoded)

## [Optional] To play further with image generation
You can try variational autoencoder model with the following dataset: </br>
1. Cat & Dog https://www.kaggle.com/c/dogs-vs-cats/data </br>
2. Celebrity image (for those who have high computational power) http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html</br>

### More on Autoencoder Model

Autoencoder tutorial: 
1. https://www.datacamp.com/community/tutorials/autoencoder-keras-tutorial
2. https://www.kaggle.com/rvislaywade/visualizing-mnist-using-a-variational-autoencoder

**Note: Post your work on the facebook using Colab to get an additional point**
